In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model

def test_model(model, test_data):
    # Initialize metrics
    precision_metric = tf.keras.metrics.Precision()
    recall_metric = tf.keras.metrics.Recall()
    accuracy_metric = tf.keras.metrics.BinaryAccuracy()
    
    # Iterate over the test data
    for x, y_true in test_data:
        y_pred = model.predict(x)
        
        # Update metrics
        precision_metric.update_state(y_true, y_pred)
        recall_metric.update_state(y_true, y_pred)
        accuracy_metric.update_state(y_true, y_pred)
    
    # Calculate final metric values
    precision = precision_metric.result().numpy()
    recall = recall_metric.result().numpy()
    accuracy = accuracy_metric.result().numpy()
    
    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)  # adding small epsilon to avoid division by zero
    
    return f"""Test results for model:
Precision: {precision:.4f}
Recall: {recall:.4f}
Binary Accuracy: {accuracy:.4f}
F1 Score: {f1_score:.4f}"""

def main():
    # Specify the folder containing the model weights
    models_folder = "path/to/your/models/folder"
    
    # Specify the folder to save the results
    results_folder = "path/to/save/results"
    
    # Specify the directory containing your test data
    data_dir = "path/to/your/test/data"
    
    # Create the results folder if it doesn't exist
    os.makedirs(results_folder, exist_ok=True)
    
    # Load your test data
    test_data = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        image_size=(224, 224),
        batch_size=32,
        shuffle=False  # Important for consistent results
    )
    
    # Iterate through all files in the models folder
    for filename in os.listdir(models_folder):
        if filename.endswith(".h5"):  # Keras models are usually saved with .h5 extension
            model_path = os.path.join(models_folder, filename)
            
            # Load the model
            model = load_model(model_path)
            
            # Test the model
            results = test_model(model, test_data)
            
            # Save the results
            result_filename = f"{os.path.splitext(filename)[0]}_results.txt"
            result_path = os.path.join(results_folder, result_filename)
            
            with open(result_path, "w") as f:
                f.write(results)
            
            print(f"Tested model {filename} and saved results to {result_filename}")

if __name__ == "__main__":
    main()